In [ ]:
!pip install -q fuzzywuzzy python-Levenshtein Levenshtein nltk pandas pyarrow seaborn

In [ ]:
PATH_TO_PARQUET="../../"
NOM_PARQUET_LABEL="Dares_accords_docx_sommaire_label_230811.parquet"
NOM_PARQUET_RESULTAT="Dares_accords_docx_sommaire_result_310811_05_2.parquet"

In [ ]:
list_of_titles=["ACCORD D'ENTREPRISE RELATIF AU\nFORFAIT ANNUEL EN J OURS",
 '- Eurofins Labazur Guyane',
 'PREAMBULE',
 "Article 1.Champ d'application",
 'Article 2.Salariés concernés',
 'Article 3.Durée annuelle du travail décomptée en jours',
 'Article 4.Conclusion d’une convention individuelle avec chaque salarié concerné',
 'Article 5.Forfait annuel en jours « réduit »',
 'Article 6.Impact des absences et arrivées/départs en cours de période',
 'Article 7.Rémunération des salariés',
 'Article 8.Garanties applicables au forfait annuel en jours et droit à la déconnexion',
 '1. Garanties sur la durée du travail',
 '2. Droit à la déconnexion',
 'Article 9.Suivi de la charge de travail et contrôle du nombre de jours travaill',
 '“ Suivi individuel et contrôle',
 '— Repos hebdomadaires ;',
 '“ Entretiens individuels et points de suivi',
 '— sa charge de travail,',
 "“ Procédure d’alerte en cas de dysfonctionnements afférents à la charge de travail, à\nl'amplitude des journées de travail et à l'équilibre entre vie privée/vie professionnelle",
 'Article 10. Dispositions finales',
 'Article 10.1 : Durée - Entrée en vigueur',
 'Article 10.2 : Révision - Dénonciation',
 'Article 10.3 : Dépôt - Publicité',
 'Pour Eurofins Labazur Guyane']

In [ ]:
import pandas as pd
import seaborn as sns
from Levenshtein import distance
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk import edit_distance
import os

In [ ]:
string2 = "Dépot - Publicité"

print(min([(distance(string1, string2),string1) for string1 in list_of_titles]))

In [ ]:
list_of_labeled_titles=[
"ACCORD D�ENTREPRISE RELATIF AU\nFORFAIT ANNUEL EN JOURS",
"PREAMBULE",
"Article 1.      Champ d�application",
"Article 2.      Salari�s concern�s",
"Article 3.      Dur�e annuelle du travail d�compt�e en jours",
"Article 4.      Conclusion d�une convention individuelle avec chaque salari� concern�",
"Article 5.      Forfait annuel en jours � r�duit �",
"Article 6.      Impact des absences et arriv�es/d�parts en cours de p�riode",
"Article 7.      R�mun�ration des salari�s",
"Article 8.      Garanties applicables au forfait annuel en jours et droit � la d�connexion",
"Article 9.      Suivi de la charge de travail et contr�le du nombre de jours travaill�s",
"Article 10. Dispositions finales",
"Article 10.1 : Dur�e - Entr�e en vigueur",
"Article 10.2 : R�vision - D�nonciation",
"Article 10.3 : D�p�t - Publicit�"
]

In [ ]:
for label in list_of_labeled_titles:
    print(min([(distance(string1, label),string1) for string1 in list_of_titles]))

In [ ]:
def compute_Levensthein_sim(str1,str2):
    edit_dist  = edit_distance(str1,str2)
    len_total = max(len(str1),len(str2))
    return ((len_total-edit_dist)/len_total)*100

In [ ]:
str1 = 'Article 8.      Garanties applicables au forfait annuel en jours et droit � la d�connexion'
str2 = 'Article 8.Garanties applicables au forfait annuel en jours et droit à la déconnexion'
edit_dist  = edit_distance(str1,str2)
print(edit_dist)
len_total = max(len(str1),len(str2))
pct_edit_dist = ((len_total-edit_dist)/len_total)*100
print(pct_edit_dist)

In [ ]:
str1 = 'Article 8.      Garanties applicables au forfait annuel en jours et droit � la d�connexion'
str2 = 'Article 8.Garanties applicables au forfait annuel en jours et droit à la déconnexion'
fuzz.ratio(str1,str2)

In [ ]:
df_label=pd.read_parquet(os.path.join(PATH_TO_PARQUET,NOM_PARQUET_LABEL))
df_result=pd.read_parquet(os.path.join(PATH_TO_PARQUET,NOM_PARQUET_RESULTAT))

In [ ]:
#df_result=df_result.drop(columns="index").set_index("num_dossier")

In [ ]:
def match_part(x,index):
    return x.str.split("-----")[0][index-1] if len(x.str.split("-----")[0])>=index  else None

In [ ]:
df_label["titre_label"]=df_label[['sommaire_label']].apply(lambda x : match_part(x,1),axis=1)
df_label["corps_label"]=df_label[['sommaire_label']].apply(lambda x : match_part(x,2) ,axis=1)
df_label["annexe_label"]=df_label[['sommaire_label']].apply(lambda x : match_part(x,3),axis=1)

In [ ]:
df_result["titre_result"]=df_result[['sommaire_result']].apply(lambda x : match_part(x,1),axis=1)
df_result["corps_result"]=df_result[['sommaire_result']].apply(lambda x : match_part(x,2) ,axis=1)

In [ ]:
df_join=df_label.join(df_result)

In [ ]:
df_join["titre_Levensthein"]=df_join[["titre_label","titre_result"]].apply(lambda x :distance(*x),axis=1)
df_join["titre_Levensthein_sim"]=df_join[["titre_label","titre_result"]].apply(lambda x :compute_Levensthein_sim(*x),axis=1)
df_join["titre_fuzz_ratio"]=df_join[["titre_label","titre_result"]].apply(lambda x :fuzz.ratio(*x),axis=1)
df_join["titre_fuzz_partial_ratio"]=df_join[["titre_label","titre_result"]].apply(lambda x :fuzz.partial_ratio(*x),axis=1)

In [ ]:
sns.displot(df_join.titre_fuzz_partial_ratio)

In [ ]:
df_join[df_join.titre_fuzz_partial_ratio<50][["titre_label","titre_result","titre_fuzz_partial_ratio"]].to_csv("my_csv.csv")

In [ ]:
def is_annexe_present(x,y):
    return ("annexe" in x.lower(),"annexe" in y.lower())

In [ ]:
df_join["presence_annexe_label_result"]=df_join[["sommaire_label","sommaire_result"]].apply(lambda x : is_annexe_present(*x),axis=1)

In [ ]:
df_join.groupby("presence_annexe_label_result")[["presence_annexe_label_result"]].count()